<a href="https://colab.research.google.com/github/DSGP-Group-1-EAPS/SL-Apparel-Dataset/blob/main/Binary_Classification_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the preprocessed data
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DSGP_COURSEWORK/SL Apparel Dataset model/preprocessed_data.xlsx')

# Features and target variable
features = ['Encoded Reason', 'Encoded Status', 'Encoded Absenteeism Type', 'Encoded Shift']
X = df[features]
y = df['Encoded Absent/Present']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Accuracy: 0.91
Confusion Matrix:
[[16423     9]
 [ 1574     9]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     16432
           1       0.50      0.01      0.01      1583

    accuracy                           0.91     18015
   macro avg       0.71      0.50      0.48     18015
weighted avg       0.88      0.91      0.87     18015



In [4]:
# Predict probabilities
y_probabilities = model.predict_proba(X_test)[:, 1]

# Identify highly likely employees (probability > 0.8)
threshold = 0.8
highly_likely_employees = df.loc[y_test.index][y_probabilities > threshold]

# Display highly likely employees
print("Highly Likely Employees:")
print(highly_likely_employees)

Highly Likely Employees:
Empty DataFrame
Columns: [Date, Shift, Code, Absenteeism Type, Status, Leave Type, Absent/Present, Reason, Encoded Reason, Encoded Absent/Present, Encoded Status, Encoded Absenteeism Type, Encoded Shift]
Index: []


##Different code

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the preprocessed data
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DSGP_COURSEWORK/SL Apparel Dataset model/preprocessed_data.xlsx')

# Create a target variable (e.g., took leave in the next month)
df['TookLeaveNextMonth'] = df['Absent/Present'].eq('Absent').shift(-1)

# Features and target variable
features = ['Encoded Reason', 'Encoded Status', 'Encoded Absenteeism Type', 'Encoded Shift']
X = df[features]
y = df['TookLeaveNextMonth']

# Handle missing values in the target variable
y = y.fillna(0)  # Fill missing values with 0 or choose an appropriate strategy

# Ensure binary classes in the target variable
y = y.astype(int)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_prob = model.predict_proba(X_test)[:, 1]

# Define probability threshold
threshold = 0.8

# Filter employees with probability above the threshold
highly_likely_employees = df.loc[X_test.index, ['Code', 'Encoded Absent/Present', 'Leave Type', 'Reason']]
highly_likely_employees['Probability'] = y_prob

# Display the result
result = highly_likely_employees[highly_likely_employees['Probability'] > threshold]
print("Highly Likely Employees:")
print(result[['Code', 'Probability']])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Highly Likely Employees:
         Code  Probability
51273  AA1020     0.995360
75297  AA2889     0.981685
49296   AA745     0.989119
40999   AA933     0.993470
70357  AA1214     0.968826
...       ...          ...
32027  AA2810     0.995579
87640  AA2925     0.987327
57120   AA603     0.974451
32529  AA3645     0.995579
72360  AA1138     1.000000

[14668 rows x 2 columns]


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the preprocessed data
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DSGP_COURSEWORK/SL Apparel Dataset model/preprocessed_data.xlsx')

# Create a target variable (e.g., took leave in the next month)
df['TookLeaveNextMonth'] = df['Absent/Present'].eq('Absent').shift(-1)

# Features and target variable
features = ['Encoded Reason', 'Encoded Status', 'Encoded Absenteeism Type', 'Encoded Shift']
X = df[features]
y = df['TookLeaveNextMonth']

# Handle missing values in the target variable
y = y.fillna(0)  # Fill missing values with 0 or choose an appropriate strategy
print(y)
# Ensure binary classes in the target variable
y = y.astype(int)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Define probability threshold
threshold = 0.8

# Filter employees with probability above the threshold
highly_likely_employees = df.loc[X_test.index, ['Code', 'Encoded Absent/Present', 'Leave Type', 'Reason']]
highly_likely_employees['Probability'] = y_prob

# Display the result
result = highly_likely_employees[highly_likely_employees['Probability'] > threshold]

# Display accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display highly likely employees
print("\nHighly Likely Employees:")
print(result[['Code', 'Probability']])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0        False
1        False
2        False
3        False
4        False
         ...  
90069     True
90070     True
90071     True
90072     True
90073        0
Name: TookLeaveNextMonth, Length: 90074, dtype: object
Accuracy: 0.91
Confusion Matrix:
[[    8  1606]
 [   10 16391]]
Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.00      0.01      1614
           1       0.91      1.00      0.95     16401

    accuracy                           0.91     18015
   macro avg       0.68      0.50      0.48     18015
weighted avg       0.87      0.91      0.87     18015


Highly Likely Employees:
         Code  Probability
51273  AA1020     0.995360
75297  AA2889     0.981685
49296   AA745     0.989119
40999   AA933     0.993470
70357  AA1214     0.968826
...       ...          ...
32027  AA2810     0.99

##Separating the datset into induvidual dataframes for each month

In [14]:
import pandas as pd

# Load the preprocessed data
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DSGP_COURSEWORK/SL Apparel Dataset model/preprocessed_data.xlsx')

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Separate the dataset into monthly sets
unique_months = pd.to_datetime(df['Date']).dt.to_period("M").unique()
monthly_datasets = {month: df[df['Date'].dt.to_period("M") == month] for month in unique_months}

# Define a function to get the encoded columns for a given month
def get_encoded_data(month_data):
    encoded_columns = ['Encoded Reason', 'Encoded Status', 'Encoded Absenteeism Type', 'Encoded Shift']
    return month_data[encoded_columns]

# Example: Access the DataFrame for December 2022 with only encoded columns
dec_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-12')])

# Repeat the above for every month
jan_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-01')])
feb_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-02')])
mar_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-03')])
apr_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-04')])
may_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-05')])
jun_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-06')])
jul_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-07')])
aug_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-08')])
sep_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-09')])
oct_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-10')])
nov_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-11')])
dec_2022_data = get_encoded_data(monthly_datasets[pd.Period('2022-12')])
jan_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-01')])
feb_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-02')])
mar_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-03')])
apr_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-04')])
may_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-05')])
jun_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-06')])
jul_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-07')])
aug_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-08')])
sep_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-09')])
oct_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-10')])
nov_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-11')])
dec_2023_data = get_encoded_data(monthly_datasets[pd.Period('2023-12')])

print(dec_2022_data)


       Encoded Reason  Encoded Status  Encoded Absenteeism Type  Encoded Shift
60064              11               1                         0              0
60065              11               1                         0              0
60066              15               1                         0              0
60067              11               1                         0              1
60068              11               1                         0              1
...               ...             ...                       ...            ...
63960               9               1                         0              1
63961              14               0                         1              1
63962              11               1                         0              0
63963              14               0                         1              1
63964               9               1                         0              1

[3901 rows x 4 columns]
